In [ ]:
# UltimateSynopticArchive VERSION 2.00
# Released: 11/16/2022

# Version 2.00 Changelog:
#    -Fixed major issues with data retrival
#        -Added in second GFS source to fill in gap in availibe data (~2007-2020)
#            -Priority is given to CFSR over old GFS data given better resolution
#        -As a result of different resolutions, added in further windslice handling for different projections
#    -Added new map type --> Theta-E and MSLP at 850mb
#    -Added new map type --> Winds and heights at 850mb
#    -Added new map type --> PWAT and 925-400mb average winds
#        -This replaces the 850mb moisture map from Version 1.00
#    -Made significant revisions to 2m temp map
#        -Changed colortable to standard NWS temp
#        -Added option to turn MSLP on and off for this map
#        -Added max/min point plotting (turns on and off w/ MSLP)
#        -Changed freezing line to be blue and dashed
#        -Removed 1000-500mb thickness (repetative when temps are already displayed)
#    -Changed sector views to better reflect identified sectors (some projection changes too!)
#    -Fixed state boundaries issue outlined in previous "known issues"
#    -Decreased state and country boundry line widths for better identification betweeen them and wx data
#    -Added tick mark kwarg to colorbars

#----------------------------------------------------------------------------------------------------------------

# *IMPORTANT* steps to using this code successfully:
#    1: Be sure to enter *ALL* required information in code cell #3 below
#    ---> This includes the save location... You may need to create a new folder where this notebook is located
#    2: *AFTER* you have entered all of the correct information, run *ALL* cells
#    3: Wait at the last code cell for prompts on successful (or unsuccessful) map creation
#    4: After prompted that the map(s) have been created, check your save folder to make sure they are there

#----------------------------------------------------------------------------------------------------------------

# Troubleshooting checklist:
#    -The code ran successfully, but my maps have no or incorrect data plotted on them
#        -Try re-running the code
#        -If the issue persists, try running a different case date (I recommend 12/28/2021 as I know it works)
#            -If changing the case date solves the issue, your case date's data may be corrupted
#    -I am getting an error during "URL Fetching"
#        -Is your case date between January 1, 1979 and a month ago? If not, data is not availible for your case
#        -If yes, then make sure your date is correctly input into code cell #3 (ie: leading zero for month/day 1-9)
#        -If the issue persists, try running a different case date (I recommend 12/28/2021 as I know it works)
#            -If changing the case date solves the issue, your case date's data may be missing
#    -I am getting an error thrown on the "plt.savefig" line
#        -Make sure your save location in code cell #3 exists (ie: you have created a folder and inserted the name in cell #3)

# Known Issues:
#    -Missing data at the Prime Meridian (Europe projection *ONLY*)
#        -Some solutions to this probelm exist online, but are not immediately compatible with PyGrib Grids
#    -925mb frontogenesis may struggle to plot for certain cases ---> Unknown cause at this time
#    -Lows/Highs on the surface map sometimes plot outside of map bounding box

# Future Plans:
#    -Add a p-type, MSLP, and thickness map
#    -Add CAPE/CIN and shear map
#    -Further develop 925mb frontogenesis map
#        -Potentially changing to layer-average (925, 850, 700mb) frontogenesis
#    -Add handling for exceedance of max and min values
#    -Add more sector view options
#        -Currently planned additions: Canada, Atlantic, and China
#    -Add handling for multiple timestep input
#    -Add support for ERA5 dataset (1950-1979 cases)
#    -Add Southern Hemisphere Views (Need to handle flipping wind barbs, switching vorticity sign)
#        -Might be easier to have a completely different notebook for Southern Hemisphere cases

# If you: 
#    -Find any bugs while using the code (not already outlined in "known issues" above)
#    -Are having problems with the code (and have went through the troubleshooting checklist above)
#    -Have suggestions for new features/map types to be added to the code
#    ---> Please let me know! Twitter is probably the best way to contact me: @KniprathWx

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pygrib
from metpy.units import units
import metpy.calc as mpcalc
import cmocean as cmo
import scipy.ndimage as ndimage
import urllib.request 
from colorama import Fore
import time
import matplotlib.colors as colors

In [ ]:
#Choose the year/month/day/hour of your case
month = '06' #Leading zero **REQUIRED** for months 1-9
day = '21' #Leading zero **REQUIRED** for days 1-9
year = '2015' #1979-Present (data not availible for up to the most recent month)
hour = '18' #00, 06, 12, or 18 

#Selct your sector view (ie: what projection you want) ---> *Only Northern Hemisphere Projections Are Currently Availible
#NOTE: First time use of a projection will take time (1 to 5 minutes) for the code to run... be patient!
SECTOR = 'CONUS' #Options are 'CONUS', 'North America', 'Europe', or 'Asia'. (Europe has missing data at the prime meridian)

#Choose which synoptic maps you would like created - enter 'Y' for yes or 'N' for no

#250mb winds and geopotential heights
jet250 = 'Y' 

#500mb cyclonic vorticity, geopotential heights, and winds
vorticity500 = 'Y' 

#850mb theta-e and MSLP (useful for front identification)
thetaE850 = 'Y' 

#850mb winds and heights (useful for LLJ identification)
winds850 = 'Y' 

#925mb frontogenesis, geopotential heights, and winds
frontogenesis925 = 'Y' 

#Precipitable water and 925-400mb average wind speed
PWAT = 'Y' 

#2m temp, freezing line, MSLP, and 1000-500mb thickness
surface = 'Y' 

#MORE OPTIONS BELOW:

#Choose if you want wind speed in mph or knots (this applies to ALL maps)
WIND = 'mph' #Enter 'mph' or 'kt'

#Choose if you want your surface map 2m tempertures in °C or °F
TEMP = 'F' #Enter 'F' or 'C' 

#Choose if you want MSLP plotted on your 2m temperature map 
MSLPchoice = 'N' #Enter 'Y' for yes or 'N' for no

#Choose the folder (within the folder you are currently in) that you want the maps to save to
saveLoc = "SaveLoc"

In [ ]:
#URL Fetching

#Quick string creation for URL fetching
date = year+month
date2 = year+month+day
date3 = year+month+day+hour

try:
    #Set the URL
    url = 'https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/analysis/'+date+'/'+date2+'/gfs_4_'+date2+'_'+hour+'00_000.grb2'

    #Set file name
    out_Name = 'GFS_'+date2+'_'+hour+'z'

    #Download the file
    urllib.request.urlretrieve(url, out_Name)
    
    #Set the model name for file name creation
    modelName = 'GFS'
    
    #Set variaible for WindSlice selction
    windModelName = 'GFS-new'
except:
    pass
    
try:    
    #Set the URL
    url = 'https://www.ncei.noaa.gov/oa/prod-cfs-reanalysis/6-hourly-by-pressure-level/'+year+'/'+date+'/'+date2+'/pgbh01.gdas.'+date3+'.grb2'
    
    #Set file name
    out_Name = 'CFSR_'+date2+'_'+hour+'z'

    #Download the file
    urllib.request.urlretrieve(url, out_Name)
    
    #Set the model name for file name creation
    modelName = 'CFSR'
    
    #Set variaible for WindSlice selction
    windModelName = 'CFSR'
except:
    #Set the URL
    url = 'https://www.ncei.noaa.gov/data/global-forecast-system/access/historical/analysis/'+date+'/'+date2+'/gfsanl_3_'+date2+'_'+hour+'00_000.grb'

    #Set file name
    out_Name = 'GFS_'+date2+'_'+hour+'z'

    #Download the file
    urllib.request.urlretrieve(url, out_Name)

    #Set the model name for file name creation
    modelName = 'GFS'
    
    #Set variaible for WindSlice selction
    windModelName = 'GFS-old'

In [ ]:
#This opens the grb file and displays all varibles present in the file
grbs = pygrib.open(out_Name)
grbs.seek(0)

for grb in grbs:
    print(grb)

In [ ]:
#Find Time & Date Variables in grb File 
T = grb.validDate

T = str(T)    

times = T[11:13]

times = "%sz" % (times)

year = T[0:4]

month = T[5:7]

day = T[8:10]

#Create strings for file names
date = "%s/%s/%s" % (month, day, year)
saveDate = "%s-%s-%s" % (month, day, year)

#Create text string for map title
validT = "Valid: %s %s" % (times, date)

#Create File Names For Later
mapNameJet250 = "250mbJet_"+modelName
fileNameJet250 = "%s/%s_%s_%s.png" % (saveLoc, mapNameJet250, saveDate, times)

mapNameVorticity500 = "500mbVorticity_"+modelName
fileNameVorticity500 = "%s/%s_%s_%s.png" % (saveLoc, mapNameVorticity500, saveDate, times)

mapNameThetaE850 = "850mbThetaE_"+modelName
fileNameThetaE850 = "%s/%s_%s_%s.png" % (saveLoc, mapNameThetaE850, saveDate, times)

mapNameLLJ850 = "850mbWinds_"+modelName
fileNameLLJ850 = "%s/%s_%s_%s.png" % (saveLoc, mapNameLLJ850, saveDate, times)

mapNameFrontogenesis925 = "925mbFrontogenesis_"+modelName
fileNameFrontogenesis925 = "%s/%s_%s_%s.png" % (saveLoc, mapNameFrontogenesis925, saveDate, times)

mapNamePWAT = "PWAT_"+modelName
fileNamePWAT = "%s/%s_%s_%s.png" % (saveLoc, mapNamePWAT, saveDate, times)

mapNameSurface = "Surface_"+modelName
fileNameSurface = "%s/%s_%s_%s.png" % (saveLoc, mapNameSurface, saveDate, times)

In [ ]:
def JetMap250():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 250mb Heights
    HGT_250mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=250)[0]
    HGT_250mb = HGT_250mb.values
    HGT_250mb = units('meter') * HGT_250mb
    
    #Data selection for U Wind Component at 250mb
    UWND_250mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    UWND_250mb = UWND_250mb.values
    UWND_250mb = units('m/s') * UWND_250mb

    #Data selection for V Wind Component at 250mb
    VWND_250mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    VWND_250mb = VWND_250mb.values
    VWND_250mb = units('m/s') * VWND_250mb
    
    #Calculate wind speed at 250mb
    sped_250mb = mpcalc.wind_speed(UWND_250mb, VWND_250mb).to(WIND)
    
    #Convert units to decameters for geopotential heights
    HGT_250mb = HGT_250mb / 10

    #Apply slight smoothing to parameters
    sped_250mb = ndimage.gaussian_filter(sped_250mb, sigma=1, order=0)
    HGT_250mb = ndimage.gaussian_filter(HGT_250mb, sigma=1, order=0)
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
    
    #Setting up the ranges for contoured wind speeds
    clevs_sped_250mb = [50, 75, 100, 125, 150, 175, 200, 225, 250]
    
    #*OLD* Custom Colortable
    #Creating a custom colormap with matplotlib colors 
    #cmap_data = ['yellow', 'orange', 'orangered', 'saddlebrown', 'darkviolet', 'deeppink', 'violet', 'lavenderblush']
    #cmapWIND = colors.ListedColormap(cmap_data, 'temperature')
    #norm = colors.BoundaryNorm(clevs_sped_250mb, cmapWIND.N)
    
    #Small function to truncate the wind speed color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('gist_ncar')
    gist_ncar_REFORMED = truncate_colormap(cmap, 0.18, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=gist_ncar_REFORMED)
    #plt.show()

    #Plot colorfill for wind speeds at 250mb
    cf = ax.contourf(lons, lats, sped_250mb, clevs_sped_250mb, cmap=cmo.tools.lighten(gist_ncar_REFORMED, 0.6), transform=ccrs.PlateCarree(), zorder=65)

    #Plot a grey contour line at transitions values for wind speeds at 250mb
    cs2 = ax.contour(lons, lats, sped_250mb, clevs_sped_250mb, colors='grey', transform=ccrs.PlateCarree(), alpha=0.6, linewidths=1, zorder=66)

    #Add colorbar for shaded wind speeds at 250mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Wind Speed ('+WIND+')')

    #Plot contours for geopotential heights at 250mb
    clevs_HGT_250mb = np.arange(0, 1200, 12)
    cs = ax.contour(lons, lats, HGT_250mb, clevs_HGT_250mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE))           
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_250mb.to(WIND)[wind_slice].m, VWND_250mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('250mb '+modelName+' Geopotential Heights (black, dam), Wind Speed (shaded, '+WIND+'), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameJet250, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def VorticityMap500():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 500mb Heights
    HGT_500mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=500)[0]
    HGT_500mb = HGT_500mb.values
    HGT_500mb = units('meter') * HGT_500mb

    #Data selection for U Wind Component at 500mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 500mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Data selection for Vorticity at 500mb
    VORT_500mb = grbs.select(name='Absolute vorticity', typeOfLevel='isobaricInhPa', level=500)[0]
    VORT_500mb = VORT_500mb.values
    VORT_500mb = units('1/s') * VORT_500mb
    
    #Convert units to decameters for geopotential heights
    HGT_500mb = HGT_500mb / 10
    
    #Convert 500mb Vorticity into usable units
    VORT_500mb = VORT_500mb * (1 / 10 ** -5)

    #Apply slight smoothing to parameters
    VORT_500mb = ndimage.gaussian_filter(VORT_500mb, sigma=1, order=0)
    HGT_500mb = ndimage.gaussian_filter(HGT_500mb, sigma=1, order=0)
    
    #Small function to truncate the vorticity color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('hot_r')
    HOT_R = truncate_colormap(cmap, 0.10, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
    #plt.show()
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12
            
    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
    
    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=1, zorder=70)    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    #Plot colorfill of 500mb vorticity
    clevs_VORT = np.arange(10,51,1)
    cf = ax.contourf(lons, lats, VORT_500mb, clevs_VORT, cmap=HOT_R, transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded vorticity at 500mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Cyclonic Vorticity (1/sec*10^-5)')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_500mb = np.arange(0, 800, 6)
    cs = ax.contour(lons, lats, HGT_500mb, clevs_HGT_500mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_500mb.to(WIND)[wind_slice].m, VWND_500mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('500mb '+modelName+' Geopotential Heights (balck, dam), Cyclonic Vorticity (shaded, 1/sec*10^-5), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameVorticity500, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def ThetaE850():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]
    
    #Data selection for temperature at 850mb
    TMP_850mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=850)[0]
    TMP_850mb = TMP_850mb.values
    TMP_850mb = units('kelvin') * TMP_850mb
    
    #Data selection for RH at 850mb
    RH_850mb = grbs.select(name='Relative humidity', typeOfLevel='isobaricInhPa', level=850)[0]
    RH_850mb = RH_850mb.values
    RH_850mb = units('%') * RH_850mb
    
        #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0] 
    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')
    
    #Calculate dew point at 850mb
    DEW_850mb = mpcalc.dewpoint_from_relative_humidity(TMP_850mb, RH_850mb)
    
    #Calculate theta-e at 850mb
    THETAE_850mb = mpcalc.equivalent_potential_temperature(850 * units.mbar, TMP_850mb, DEW_850mb)
    
    #Apply slight smoothing to parameters
    THETAE_850mb = ndimage.gaussian_filter(THETAE_850mb, sigma=1, order=0)
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
        
    #Setting up the ranges for contoured theta-e
    clevs_THETAE = np.arange(240, 381, 2)

    #Creating a custom colortable using RGB values
    cmap_data = [(50/255,50/255,50/255),(73/255,73/255,73/255),(97/255,97/255,97/255),(121/255,121/255,121/255),(145/255,145/255,145/255),(182/255,168/255,186/255),(178/255,153/255,199/255),(187/255,141/255,227/255),(170/255,101/255,230/255),(154/255,67/255,230/255),(112/255,43/255,171/255),(81/255,29/255,125/255),(58/255,23/255,87/255),(23/255,23/255,87/255),(49/255,49/255,97/255),(49/255,49/255,121/255),(49/255,49/255,137/255),(49/255,49/255,153/255),(49/255,49/255,169/255),(49/255,49/255,193/255),(49/255,49/255,220/255),(49/255,49/255,251/255),(74/255,74/255,253/255),(81/255,83/255,246/255),(90/255,92/255,246/255),(110/255,112/255,245/255),(130/255,132/255,246/255),(151/255,153/255,245/255),(171/255,173/255,245/255),(192/255,194/255,245/255),(212/255,214/255,246/255),(233/255,235/255,246/255),(222/255,240/255,224/255),(182/255,229/255,184/255),(141/255,220/255,143/255),(59/255,199/255,61/255),(141/255,220/255,42/255),(209/255,236/255,42/255),(243/255,237/255,42/255),(243/255,223/255,42/255),(253/255,212/255,49/255),(252/255,201/255,49/255),(252/255,187/255,49/255),(252/255,171/255,49/255),(252/255,157/255,49/255),(252/255,139/255,49/255),(252/255,122/255,49/255),(252/255,104/255,49/255), (252/255,85/255,49/255), (253/255,49/255,49/255), (234/255,42/255,42/255), (205/255,49/255,49/255),(172/255,42/255,42/255),(140/255, 42/255, 42/255),(110/255, 35/255, 35/255),(78/255, 35/255, 35/255),(78/255, 35/255, 61/255),(115/255, 52/255, 90/255),(156/255, 75/255, 123/255), (173/255, 71/255, 132/255),(214/255, 60/255, 152/255),(224/255, 119/255, 223/255),(237/255, 164/255, 236/255),(237/255, 197/255, 237/255),(214/255, 197/255, 214/255), (237/255, 225/255, 237/255),(245/255, 218/255, 218/255),(214/255, 169/255, 169/255),(150/255, 113/255, 113/255),(112/255, 82/255, 82/255)]
    cmap = colors.ListedColormap(cmap_data, 'temperature')
    norm = colors.BoundaryNorm(clevs_THETAE, cmap.N)

    #Plotting theta-e with colorfill
    cf = ax.contourf(lons, lats, THETAE_850mb, clevs_THETAE, cmap=cmo.tools.lighten(cmap, 0.7), transform=ccrs.PlateCarree(), zorder=65)

    #Creating a colorbar for theta-e
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[240, 260, 280, 300, 320, 340, 360, 380])
    cb.set_label('Equivalent Potential Temperature (K)')

    #Plot a grey contour line at transition values for theta-e
    cs22 = ax.contour(lons, lats, THETAE_850mb, clevs_THETAE, linewidths=0.75, colors='grey', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    #Countoring MSLP
    cs3 = ax.contour(lons, lats, MSLP, range(900, 1050, 2), transform=ccrs.PlateCarree(), colors='k', linestyles='solid', zorder=77)
    cs3.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)
    
    #Add titles
    plt.title('850mb '+modelName+' Theta-E (shaded, K) and MSLP (black, hPa)', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameThetaE850, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def LLJMap850():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 850mb Heights
    HGT_850mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=850)[0]
    HGT_850mb = HGT_850mb.values
    HGT_850mb = units('meter') * HGT_850mb
    
    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb
    
    #Calculate wind speed at 250mb
    sped_850mb = mpcalc.wind_speed(UWND_850mb, VWND_850mb).to(WIND)
    
    #Convert units to decameters for geopotential heights
    HGT_850mb = HGT_850mb / 10

    #Apply slight smoothing to parameters
    sped_850mb = ndimage.gaussian_filter(sped_850mb, sigma=1, order=0)
    HGT_850mb = ndimage.gaussian_filter(HGT_850mb, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()
    
    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12
            
    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
    
    #Setting up the ranges for contoured wind speeds
    clevs_sped_850mb = [30, 40, 50, 60, 70, 80, 100, 125, 150]
    
    #Creating a custom colormap with matplotlib colors 
    cmap_data = ['yellow', 'orange', 'orangered', 'saddlebrown', 'darkviolet', 'deeppink', 'violet', 'lavenderblush']
    cmapWIND = colors.ListedColormap(cmap_data, 'temperature')
    norm = colors.BoundaryNorm(clevs_sped_850mb, cmapWIND.N)

    #Plot colorfill for wind speeds at 850mb
    cf = ax.contourf(lons, lats, sped_850mb, clevs_sped_850mb, cmap=cmo.tools.lighten(cmapWIND, 0.6), transform=ccrs.PlateCarree(), zorder=65)

    #Plot a grey contour line at transitions values for wind speeds at 250mb
    cs2 = ax.contour(lons, lats, sped_850mb, clevs_sped_850mb, colors='grey', transform=ccrs.PlateCarree(), alpha=0.6, linewidths=1, zorder=66)

    #Add colorbar for shaded wind speeds at 850mb
    cb = plt.colorbar(cf, orientation='horizontal', spacing='proportional', pad=0, aspect=50)
    cb.set_label('Wind Speed ('+WIND+')')

    #Plot contours for geopotential heights at 850mb
    clevs_HGT_850mb = np.arange(0, 800, 5)
    cs = ax.contour(lons, lats, HGT_850mb, clevs_HGT_850mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE))           
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_850mb.to(WIND)[wind_slice].m, VWND_850mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('850mb '+modelName+' Geopotential Heights (black, dam), Wind Speed (shaded, '+WIND+'), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameLLJ850, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def FrontogenesisMap925():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]
    
    #Data selection for 925mb Heights
    HGT_925mb = grbs.select(name='Geopotential Height', typeOfLevel='isobaricInhPa', level=925)[0]
    HGT_925mb = HGT_925mb.values
    HGT_925mb = units('meter') * HGT_925mb
    
    #Data selection for 925mb Temperatures
    TEMP_925mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=925)[0]
    TEMP_925mb = TEMP_925mb.values
    TEMP_925mb = units('kelvin') * TEMP_925mb
    
    #Data selection for U Wind Component at 925mb
    UWND_925mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    UWND_925mb = UWND_925mb.values
    UWND_925mb = units('m/s') * UWND_925mb

    #Data selection for V Wind Component at 925mb
    VWND_925mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    VWND_925mb = VWND_925mb.values
    VWND_925mb = units('m/s') * VWND_925mb
    
    #Setting variables for calculating frontogenesis
    level925 = 925 * units.hPa
    lats, lons = grb.latlons()
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    #Calculating theta at 925mb for frontogenesis calculation
    THETA_925mb = mpcalc.potential_temperature(level925, TEMP_925mb)

    #Calculating frontogenesis
    FRONTO_925mb = mpcalc.frontogenesis(THETA_925mb, UWND_925mb, VWND_925mb, dx, dy)
    convert_to_per_100km_3h = 1000*100*3600*3
    FRONTO_925mb = FRONTO_925mb * convert_to_per_100km_3h
    
    #Apply slight smoothing to parameters
    #FRONTO_925mb = ndimage.gaussian_filter(FRONTO_925mb, sigma=1, order=0) * units('K/hour')
    HGT_925mb = ndimage.gaussian_filter(HGT_925mb, sigma=1, order=0) * units.meter
    
    #Convert heights from m to dam
    HGT_925mb = HGT_925mb / 10
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
        
    #Plot colorfill of 925mb Frontogenesis
    clevs_FRONTO = np.arange(0.5,15.5,0.5)
    cf = ax.contourf(lons, lats, FRONTO_925mb, clevs_FRONTO, cmap='cool', transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded Frontogenesis at 925mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[0.5, 3, 6, 9, 12, 15])
    cb.set_label('Frontogenesis (°C/100km/3hr)')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_925mb = np.arange(0, 300, 3)
    cs = ax.contour(lons, lats, HGT_925mb, clevs_HGT_925mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_925mb.to(WIND)[wind_slice].m, VWND_925mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('925mb '+modelName+' Frontogenesis (shaded, °C/100km/3hr), Heights (black, dam), and Wind Barbs ('+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameFrontogenesis925, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def PWATmap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for PWATs
    if modelName == 'GFS':
        PWAT = grbs.select(name='Precipitable water')[0]
    else:
        PWAT = grbs.select(name='Precipitable water', typeOfLevel="atmosphereSingleLayer")[0]
    PWAT = PWAT.values
    PWAT = units('mm') * PWAT
    PWAT = PWAT.to('inch')
    
    #Data selection for U Wind Component at 250mb
    UWND_925mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    UWND_925mb = UWND_925mb.values
    UWND_925mb = units('m/s') * UWND_925mb

    #Data selection for V Wind Component at 250mb
    VWND_925mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    VWND_925mb = VWND_925mb.values
    VWND_925mb = units('m/s') * VWND_925mb
    
    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb
    
    #Data selection for U Wind Component at 250mb
    UWND_700mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=700)[0]
    UWND_700mb = UWND_700mb.values
    UWND_700mb = units('m/s') * UWND_700mb

    #Data selection for V Wind Component at 250mb
    VWND_700mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=700)[0]
    VWND_700mb = VWND_700mb.values
    VWND_700mb = units('m/s') * VWND_700mb
    
    #Data selection for U Wind Component at 500mb
    UWND_600mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=600)[0]
    UWND_600mb = UWND_600mb.values
    UWND_600mb = units('m/s') * UWND_600mb

    #Data selection for V Wind Component at 500mb
    VWND_600mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=600)[0]
    VWND_600mb = VWND_600mb.values
    VWND_600mb = units('m/s') * VWND_600mb
    
    #Data selection for U Wind Component at 500mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 500mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Data selection for U Wind Component at 400mb
    UWND_400mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=400)[0]
    UWND_400mb = UWND_400mb.values
    UWND_400mb = units('m/s') * UWND_400mb

    #Data selection for V Wind Component at 400mb
    VWND_400mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=400)[0]
    VWND_400mb = VWND_400mb.values
    VWND_400mb = units('m/s') * VWND_400mb
    
    #Calculate average wind speed in the column
    UWND_AVG = (UWND_925mb + UWND_850mb + UWND_700mb + UWND_600mb + UWND_500mb + UWND_400mb) / 6
    VWND_AVG = (VWND_925mb + VWND_850mb + VWND_700mb + VWND_600mb + VWND_500mb + VWND_400mb) / 6
    
    #Calculate wind speed at 250mb
    sped_AVG = mpcalc.wind_speed(UWND_AVG, VWND_AVG).to(WIND)
    
    #Apply slight smoothing to parameters
    sped_AVG = ndimage.gaussian_filter(sped_AVG, sigma=1, order=0)

    #Small function to truncate the dew point color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = cmo.cm.phase_r
    PHASEcmo_R = truncate_colormap(cmap, 0, 0.70)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=PHASEcmo_R)
    #plt.show()
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()
    
    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Plot colorfill of 925mb Frontogenesis
    clevs_PWAT = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]

    cf = ax.contourf(lons, lats, PWAT, clevs_PWAT, cmap=cmo.tools.lighten(PHASEcmo_R, 0.85), transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded Frontogenesis at 925mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, spacing='proportional', ticks=[0, 0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3])
    cb.set_label('Precipitable Water (inch)')
    
    #Plot a grey contour line at transition values for 2m temperatures
    cs22 = ax.contour(lons, lats, PWAT, clevs_PWAT, linewidths=0.70, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_AVG.to(WIND)[wind_slice].m, VWND_AVG[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)
    
    #Add titles
    plt.title(modelName+' Precipitable Water (shaded, in) and 925-400mb Average Wind Speed (barbs, '+WIND+')', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNamePWAT, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def SurfaceMap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0] 
    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')

    #Data selection for MSLP
    SFC_temp = grbs.select(name='2 metre temperature')[0]
    SFC_temp = SFC_temp.values
    SFC_temp = units('K') * SFC_temp
    if TEMP == 'F':
        SFC_temp = SFC_temp.to('degF')
    else:
        SFC_temp = SFC_temp.to('degC')

    #Apply slight smoothing to parameters
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)
    SFC_temp = ndimage.gaussian_filter(SFC_temp, sigma=1, order=0)
    
    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    datacrs = ccrs.PlateCarree()
    fig = plt.figure(1, figsize=(14, 12))
    ax = plt.subplot(1, 1, 1, projection=datacrs )

    if SECTOR == 'CONUS':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-127.5, -64.5, 20, 55], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'North America':
        datacrs = ccrs.LambertConformal()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-148, -48, 8, 74], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 11
        elif windModelName == 'GFS-old':
            WINDSLICE = 5
        else:
            WINDSLICE = 11

    elif SECTOR == 'Europe':
        datacrs = ccrs.Robinson()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([-20, 40.5, 35, 72], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 6
        elif windModelName == 'GFS-old':
            WINDSLICE = 3
        else:
            WINDSLICE = 6

    elif SECTOR == 'Asia':
        datacrs = ccrs.PlateCarree()
        fig = plt.figure(1, figsize=(14, 12))
        ax = plt.subplot(1, 1, 1, projection=datacrs )
        ax.set_extent([27, 180, 5, 78.2], ccrs.PlateCarree())
        if windModelName == 'GFS-new':
            WINDSLICE = 12
        elif windModelName == 'GFS-old':
            WINDSLICE = 6
        else:
            WINDSLICE = 12

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass
        
    #If-else statement for temperture plotting °F vs °C
    if TEMP == 'F':
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-60, 121, 5)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-60, 121, 5), cmap=cmo.tools.lighten(cmap, 0.75), transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])
        cb.set_label('2m Temperature (°F)')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-60, 121, 5), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=datacrs, zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(32, 33, 1), linewidths=1.5, colors='blue', alpha=1, linestyles='dashed', transform=ccrs.PlateCarree(), zorder=71)
    else:
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-51, 58, 3)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-51, 58, 3), cmap=cmo.tools.lighten(cmap, 0.75), transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-45,-30, -15, 0, 15, 30, 45])
        cb.set_label('2m Temperature (°C)')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-51, 58, 3), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(0, 1, 1), linewidths=1.5, colors='blue', alpha=1, linestyles='dashed', transform=ccrs.PlateCarree(), zorder=71)

    #Funtion for plotting Highs and Lows ]
    def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                           plotValue=True, transform=None):
        
        from scipy.ndimage.filters import maximum_filter, minimum_filter

        if (extrema == 'max'):
            data_ext = maximum_filter(data, nsize, mode='nearest')
        elif (extrema == 'min'):
            data_ext = minimum_filter(data, nsize, mode='nearest')
        else:
            raise ValueError('Value for hilo must be either max or min')

        mxy, mxx = np.where(data_ext == data)

        for i in range(len(mxy)):
            ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
                    clip_on=True, horizontalalignment='center', verticalalignment='center',
                    transform=transform, zorder=95)
            ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                    '\n' + str(int(data[mxy[i], mxx[i]])),
                    color=color, size=12, clip_on=True, fontweight='bold',
                    horizontalalignment='center', verticalalignment='top', transform=transform, zorder=95)
    
    #Countoring MSLP (if, else from choice in settings)
    if MSLPchoice == 'Y':
        cs3 = ax.contour(lons, lats, MSLP, range(900, 1050, 4), transform=ccrs.PlateCarree(), colors='k', linestyles='solid', zorder=77)
        cs3.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)
        plot_maxmin_points(lons, lats, MSLP, 'max', 25, symbol='H', color='b',  transform=ccrs.PlateCarree())
        plot_maxmin_points(lons, lats, MSLP, 'min', 25, symbol='L', color='r', transform=ccrs.PlateCarree())
    else:
        pass
    
    #Add titles
    if TEMP == 'F' and MSLPchoice == 'Y':
        plt.title(modelName+' 2m Temp (shaded, °F), Freezing Line (dashed blue, 32°F), and MSLP (black, hPa)', loc='left')
    elif TEMP == 'F' and MSLPchoice == 'N':
        plt.title(modelName+' 2m Temp (shaded, °F) and Freezing Line (dashed blue, 32°F)', loc='left')
    elif TEMP == 'C' and MSLPchoice == 'Y':
        plt.title(modelName+' 2m Temp (shaded, °C), Freezing Line (dashed blue, 0°C), MSLP (black, hPa), and Highs/Lows (red/blue, hPa)', loc='left')
    else:
        plt.title(modelName+' 2m Temp (shaded, °C) and Freezing Line (dashed blue, 0°C)', loc='left')
    props = dict(boxstyle='round', facecolor='white')
    ax.text(0.828, 0.9825, validT, transform=ax.transAxes, fontsize=11.5, verticalalignment='top', bbox=props, zorder=100)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameSurface, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
#Handling map creation with if-then statements
print(Fore.BLUE + 'Map Creation Sequence Initiated')
print('--------------------------------------------')
time.sleep(1.5)
if jet250 == 'Y':
    JetMap250()
    print(Fore.BLACK + '250mb Jet Map Successfully Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + '250mb Jet Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'
    time.sleep(1.5)
    
if vorticity500 == 'Y':
    VorticityMap500()
    print(Fore.BLACK + '500mb Vorticity Successfully Map Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + '500mb Vorticity Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'
    time.sleep(1.5)
    
if thetaE850 == 'Y':
    ThetaE850()
    print(Fore.BLACK + '850mb Theta-E Map Successfully Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + '850mb Theta-E Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'
    time.sleep(1.5)

if winds850 == 'Y':
    LLJMap850()
    print(Fore.BLACK + '850mb Winds Map Successfully Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + '850mb Winds Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------' 
    time.sleep(1.5)

if frontogenesis925 == 'Y':
    FrontogenesisMap925()
    print(Fore.BLACK + '925mb Frontogenesis Map Successfully Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + '925mb Frontogenesis Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'
    time.sleep(1.5)

if PWAT == 'Y':
    PWATmap()
    print(Fore.BLACK + 'PWAT Map Successfully Created')
    print('--------------------------------------------')
    time.sleep(1.5)
else:
    print(Fore.RED + 'PWAT Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'  
    time.sleep(1.5)
    
if surface == 'Y':
    SurfaceMap()
    print(Fore.BLACK + 'Surface Map Successfully Created')
    print('--------------------------------------------')
else:
    print(Fore.RED + 'Surface Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------'

In [ ]:
# Some errors (such as division by 0) are normal. 
# As long as it says "map created" and the map looks fine, there is no need to be concerned.